In [71]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score,cross_validate
from numpy import mean
from numpy import std
from sklearn.metrics import mean_squared_error

In [72]:
X_train = pd.read_csv('../data/processed/X_train', header=0, index_col=0)
X_test = pd.read_csv('../data/processed/X_test', header=0, index_col=0)
y_train = pd.read_csv('../data/processed/y_train', header=0, index_col=0)
y_test = pd.read_csv('../data/processed/y_test', header=0, index_col=0)

# Feature Scaling

In [73]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
print(X_train)
print(X_test)

# LogisticRegression Model Building

In [75]:
# Training the Logistic Regression model on the Training set
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)
# Predicting the Test set results
y_pred = classifier.predict(X_test)
# Making the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy:', accuracy_score(y_test, y_pred)*100,'%')
print(classification_report(y_test,y_pred))

[[32  4]
 [ 4 20]]
Accuracy: 86.66666666666667 %
              precision    recall  f1-score   support

           0       0.89      0.89      0.89        36
           1       0.83      0.83      0.83        24

    accuracy                           0.87        60
   macro avg       0.86      0.86      0.86        60
weighted avg       0.87      0.87      0.87        60



C:\Users\Asus\PycharmProjects\Heart-Disease-Machine-Learning-Exploration\venv\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Cross Fold Validation

In [76]:
cv = KFold(n_splits=10, random_state=42, shuffle=True)
model = LogisticRegression()
print(cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1).mean())
#print(cross_val_score(model, X_test, y_test, scoring='accuracy', cv=cv, n_jobs=-1).mean())
#print("Test set:", cross_val_score(y_test, model.predict(X_test),scoring='accuracy'))

0.8193840579710144


In [ ]:
scoring = "neg_root_mean_squared_error"
model = LogisticRegression()
scores = cross_validate(model, X_train, y_train, scoring=scoring, return_estimator=True)
print("Linear regression score:", scores["test_score"].mean())

In [ ]:
import sklearn
model = sklearn.base.clone(model)
model.fit(X_train, y_train)
print("Test set RMSE:", mean_squared_error(y_test, model.predict(X_test), squared=False))
print("Mean validation RMSE:", -scores["test_score"].mean())